In [144]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import sys

In [145]:
!pip install folium
import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans
import urllib, json

In [146]:
##SCRAPPING DISTRICS

#define headers for the request
headers = requests.utils.default_headers()
headers.update({"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"})

#Wikipedia link for the information to scrap
url = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [147]:
#Finding columns name html style
columns_name = str(soup.center.table.tbody.tr).replace('\n','')

#cleaning tags
columns_string=re.findall('<th>(.+?)<',columns_name)

columns = [list(columns_string)[1],list(columns_string)[3]]
#Transform in a Series to create data frame
columns_name = pd.Series(list(columns))

In [148]:
#finding dataframe values
values = str(soup.center.find_all("td")).replace('\n','')
values = re.findall('>(.+?)</td>',values)

In [149]:
#finding Neighbourhood clean name
neigh = []
for value in values:
    if len(re.findall('>(.+?)</a>',str(value)))<1:
        neigh.append(value)
    else:
        a = re.findall('>(.+?)<',str(value)) #result type is a list
        neigh.append(a[0])


In [298]:
#Define two empty list to get District Name and population
d=[]
h1=[]
h2=[] #h1 will return a nest list, to extract the population in just one list
i=0

for i in range(int(len(neigh)/7)):
    if i==0:
        d.append(neigh[i+1])
        h1.append(re.findall('>(.+?)</span>',str(neigh[i+3])))
        h2.append(int(str(h1[i][i]).replace(',','')))
    else:
        d.append(neigh[i*7+1])
        h1.append(re.findall('>(.+?)</span>',neigh[i*7+3]))
        h2.append(int(str(h1[i][0]).replace(',','')))

        #Creating the df wich contains info about Madrid districts

dicti = {'District Name':d,
         'District Population':h2    
}

mddist = pd.DataFrame(dicti)
mddist.head(10)


,District Name,District Population
0,Centro,131928
1,Arganzuela,151965
2,Retiro,118516
3,Salamanca,143800
4,Chamartín,143424
5,Tetuán,153789
6,Chamberí,137401
7,Fuencarral-El Pardo,238756
8,Moncloa-Aravaca,116903
9,Latina,233808


In [291]:
#Conecting with Comunidad de Madrid official data to get neighborhoods

#Wikipedia link for the information to scrap
url2 = "https://datos.comunidad.madrid/catalogo/dataset/91857345-056b-4aa7-96f5-6310dac619bf/resource/b9cf6d43-79b0-400e-82de-ca00de21f6bb/download/barrios_municipio_madrid.json"
req2 = requests.get(url2, verify=False)
#With the mdjson and District name We can find the Neighborhood
mdjson = req2.json()

mdjson
#mdjson has spaces leading and ending in barrio_nombre and distrito_nombre, let's clean that


for i in range(len(mdjson['data'])):
        for j in mdjson['data'][i]:
            if type(mdjson['data'][i][j])==str:
                mdjson['data'][i][j]=mdjson['data'][i][j].strip()


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [281]:
#Now we create 2 empty list, to append distrito_nombre(District Name) & barrio_nombre(Neighborhood) 
#also we use madrid['District Name'] to go through the json

dist = mddist['District Name']
d = [] #district
n = [] #neighborhood
i=0
j=0
for i in range(len(dist)):
    for j in range (len(mdjson['data'])):
        if mdjson['data'][j]['distrito_nombre']==dist[i]:
            d.append(dist[i])
            n.append(mdjson['data'][j]['barrio_nombre'])

#lets create a 2nd df with the new info

mdneigh = pd.DataFrame({'District Name':d,
                       'Neighborhood':n
})
mdneigh.head(10)

,District Name,Neighborhood
0,Centro,Palacio
1,Centro,Embajadores
2,Centro,Cortes
3,Centro,Justicia
4,Centro,Universidad
5,Centro,Sol
6,Arganzuela,Imperial
7,Arganzuela,Las Acacias
8,Arganzuela,La Chopera
9,Arganzuela,Legazpi


In [292]:
#Now lets join df madrid with mdneigh 
madrid = pd.merge(mdneigh,mddist,on='District Name')
madrid.head(10)
#Extract data from RentaxNeighborhood.csv from github repository
madridrent = pd.read_csv('https://raw.githubusercontent.com/cazuaje27/BattleofNeighborhoods/master/RentaxNeighborhood.csv', sep=';',skiprows=2,header=1)
madridrent.rename(columns={'Territorio':'Neighborhood','Renta neta media por hogar':'Net Income by Home'}, inplace=True)

#Merge to get all Rent in madrid dataframe

madrid = madrid.merge(madridrent, on=['Neighborhood'], how='inner')
madrid.drop(axis=1, columns=['Orden','Cod. Territorio'], inplace=True)
madrid.head(10)

,District Name,Neighborhood,District Population,Net Income by Home
0,Centro,Palacio,131928,34.676
1,Centro,Embajadores,131928,26.000
2,Centro,Cortes,131928,34.953
3,Centro,Justicia,131928,40.315
4,Centro,Universidad,131928,30.702
5,Centro,Sol,131928,30.936
6,Arganzuela,Imperial,151965,41.951
7,Arganzuela,Legazpi,151965,48.935
8,Arganzuela,Delicias,151965,39.648
9,Arganzuela,Palos de Moguer,151965,33.938


-113187756.121

,District Name,Neighborhood,District Population,Orden,Cod. Territorio,Renta neta media por hogar
0,Centro,Palacio,"131,928",69,28079BAR011,34.676
1,Centro,Embajadores,"131,928",112,28079BAR012,26.000
2,Centro,Cortes,"131,928",68,28079BAR013,34.953
3,Centro,Justicia,"131,928",56,28079BAR014,40.315
4,Centro,Universidad,"131,928",84,28079BAR015,30.702
5,Centro,Sol,"131,928",83,28079BAR016,30.936
6,Arganzuela,Imperial,"151,965",50,28079BAR021,41.951
7,Arganzuela,Legazpi,"151,965",31,28079BAR024,48.935
8,Arganzuela,Delicias,"151,965",59,28079BAR025,39.648
9,Arganzuela,Palos de Moguer,"151,965",73,28079BAR026,33.938
